#####  Lib import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from importlib import reload
import seaborn as sns
%load_ext autoreload
%autoreload 2
%matplotlib inline



In [2]:
# Models
import xgboost as xgb
from xgboost import XGBRanker

# Cross-val
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
# Metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score, average_precision_score

# Processing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ndcg_score
# Feature selection
from sklearn.feature_selection import VarianceThreshold, SelectKBest, SelectFromModel

##### Data import

In [48]:
mydata = pd.read_csv(r'Data\mydata.csv')
print(mydata['srch_id'].nunique())
#mydata= mydata.set_index('srch_id')
mydata['prop_booking_bool']=mydata['prop_booking_bool']*100

24013


##### Train

In [4]:
# RandomState
random_state = 1
int(0.8*mydata['srch_id'].nunique())

19210

In [5]:
import random
random_state = 1
train_search_ID= random.sample(set(mydata['srch_id']), k= int(0.8*mydata['srch_id'].nunique()))
train_data=mydata.loc[mydata['srch_id'].isin(train_search_ID)]
test_data=mydata.loc[~mydata['srch_id'].isin(train_search_ID)]
train_data['srch_id'].nunique()+test_data['srch_id'].nunique()

Dataforevaluating = test_data.copy()



train_data= train_data.set_index('srch_id')
test_data= test_data.set_index('srch_id')

In [6]:
# X_train = train_data.drop('prop_booking_bool', axis=1).copy()
# X_test  = test_data.drop('prop_booking_bool', axis=1).copy()

# #X_train = X_train.drop('prop_key', axis=1).copy()
# #X_test  = X_test.drop('prop_key', axis=1).copy()

# y_train = train_data['prop_booking_bool'].copy()
# y_test  = test_data['prop_booking_bool'].copy()

In [26]:
X_test

,srch_visitor_visit_nbr,prop_key,prop_travelad_bool,prop_dotd_bool,prop_price_with_discount_usd,prop_imp_drr,prop_brand_bool,search_rank,price_ratio,minornot,...,prop_Price_rank,prop_meanPrice_rank,Diff_PropSTR_SubmarketSTR,starrating_floor,Rank_by_StarRating_&_price,Rank_by_StarRating_&_Bookingrate,Submarket_price_mean,Diff_PropPrice_SubmarketPrice,Submarket_Br_mean,Diff_PropSTR_SubmarketSTR_BR
srch_id,,,,,,,,,,,,,,,,,,,,,
-1046322713,5,257690,0,0,111.80,0,1,1,0.433754,0,...,16,23,0.380146,3,39,38,610.749537,498.949537,0.043650,0.023783
-1046322713,5,3066218,0,0,92.24,1,0,2,0.357866,0,...,27,14,-0.119854,3,28,15,610.749537,518.509537,0.043650,-0.041281
-1046322713,5,2271987,0,0,93.98,0,1,3,0.364617,0,...,26,12,0.380146,3,29,28,610.749537,516.769537,0.043650,-0.005732
-1046322713,5,3308025,0,0,173.30,0,1,4,0.672357,0,...,5,8,1.443147,5,1,1,541.765860,368.465860,0.034334,-0.026272
-1046322713,5,3222046,0,0,86.25,0,0,5,0.334627,0,...,29,17,-0.119854,3,27,18,610.749537,524.499537,0.043650,-0.028784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964783409,5,249748,0,0,284.00,1,1,49,0.291282,0,...,27,39,0.607190,4,9,14,282.523118,-1.476882,0.024787,-0.008211
964783409,5,255778,0,0,269.00,0,1,50,0.275897,0,...,29,4,0.115180,4,7,11,400.365371,131.365371,0.033202,-0.005562
964783409,5,244678,0,0,225.00,0,0,51,0.230769,0,...,37,52,-0.884820,3,38,40,400.365371,175.365371,0.033202,-0.007197


In [8]:
def get_ndcg_test(Eval_Data, RANK_MODEL):
    df = Eval_Data[['srch_id', 'prop_key']].reset_index(drop=True).copy() 
    srch_IDs = Eval_Data['srch_id'].unique()

    Eval_Data_indexed= Eval_Data.set_index('srch_id').copy()
    X_test_eval  = Eval_Data_indexed.drop('prop_booking_bool', axis=1).copy()
    y_train_eval = Eval_Data_indexed['prop_booking_bool'].copy()
    ndcg=[]
    
    for search_id in srch_IDs:
        df.loc[df['srch_id'] == search_id, 'rank'] = RANK_MODEL.predict(X_test_eval.loc[X_test_eval.index == search_id])
        relevance = np.asarray([list(RANK_MODEL.predict(X_test_eval.loc[X_test_eval.index == search_id]))])
        truth = np.asarray([list(Eval_Data.loc[Eval_Data['srch_id'] == search_id]['prop_booking_bool'])])
        if len(truth[0])<2:
            ndcg.append(1)
            continue
        ndcg.append(ndcg_score(truth, relevance))
    return np.nanmean(ndcg)

In [9]:
def train_xgb_model(model, X_train, y_train, Dataforevaluating, group, eval_group):
    model.fit(X_train, y_train, group, eval_set=[(X_train, y_train)], eval_group=eval_group)
    y_train_pred = model.predict(X_train)
    res_train = list(model.evals_result['eval_0'].values())[0]
    ndcg_train = round(sum(res_train)/len(res_train), 2)
    print('*'*20 + 'TRAIN' + '*'*20)
    print('train avg precision: %.2f' % (average_precision_score(y_train, y_train_pred, average='weighted')))
    print('train avg ndcg@50: %.2f' % ndcg_train)
    
    print('*'*20 + 'TEST' + '*'*20)
    print("The NDCG for the test Dataset is:",get_ndcg_test(Dataforevaluating,model))

    
    return model

In [46]:
#def get_ndcg_test(Eval_Data, RANK_MODEL):
Eval_Data= Dataforevaluating.copy()

df = Eval_Data[['srch_id', 'prop_key']].reset_index(drop=True).copy() 
srch_IDs = Eval_Data['srch_id'].unique()

Eval_Data_indexed= Eval_Data.set_index('srch_id').copy()
X_test_eval  = Eval_Data_indexed.drop('prop_booking_bool', axis=1).copy()
y_train_eval = Eval_Data_indexed['prop_booking_bool'].copy()
ndcg=[]
ratio = 0.9
RANK_MODEL= xgb_model
srch_IDs=[964783409]
n=2



for search_id in srch_IDs:
    x_test_serach=X_test_eval.loc[X_test_eval.index == search_id]
    joined=x_test_serach.copy()
    for i in range(n):
        #x_test_serach=X_test_eval.loc[X_test_eval.index == search_id]
        Len=len(x_test_serach)
        if Len>2:
            x_test_selected= x_test_serach.sample(frac = 0.9) 
            name= "rank_"+str(i)
            
            x_test_selected[name] = RANK_MODEL.predict(x_test_selected)
            
            
            
            
            joined = pd.merge(joined, x_test_selected[['prop_key',name]], on=['prop_key'], how='left')
            
            
            #prop_search_ID= random.sample(set(x_test_serach['prop_key']), k= int(ratio*x_test_serach['prop_key'].nunique()))
            #x_test_selected=x_test_serach.loc[x_test_serach['prop_key'].isin(prop_search_ID)]
            
            #name= "rank_"+str(i)
            #df.loc[(df['srch_id'] == search_id) , name]= 'asb'
            #df.loc[(df['srch_id'] == search_id) & (df['prop_key'] == x_test_selected['prop_key']) , name] = RANK_MODEL.predict(x_test_selected)
#        relevance = np.asarray([list(RANK_MODEL.predict(X_test_eval.loc[X_test_eval.index == search_id]))])
            #truth = np.asarray([list(Eval_Data.loc[Eval_Data['srch_id'] == search_id]['prop_booking_bool'])])
            #if len(truth[0])<2:
            #    ndcg.append(1)
            #    continue
            #ndcg.append(ndcg_score(truth, relevance))
#return np.nanmean(ndcg)
joined

,srch_visitor_visit_nbr,prop_key,prop_travelad_bool,prop_dotd_bool,prop_price_with_discount_usd,prop_imp_drr,prop_brand_bool,search_rank,price_ratio,minornot,...,Diff_PropSTR_SubmarketSTR,starrating_floor,Rank_by_StarRating_&_price,Rank_by_StarRating_&_Bookingrate,Submarket_price_mean,Diff_PropPrice_SubmarketPrice,Submarket_Br_mean,Diff_PropSTR_SubmarketSTR_BR,rank_0,rank_1
0,5,576979,0,0,269.0,0,1,1,0.275897,0,...,-0.326668,3,43,46,397.149000,128.149000,0.026574,0.012128,-0.500139,-0.500139
1,5,1108480,0,0,409.0,0,1,2,0.419487,0,...,0.673332,4,30,31,397.149000,-11.851000,0.026574,0.016504,-1.418964,-1.418964
2,5,3888657,0,0,975.0,1,1,3,1.000000,0,...,1.073995,5,3,3,485.716843,-489.283157,0.023606,0.022003,-4.380408,-4.380408
3,5,3407116,0,0,484.0,0,1,4,0.496410,0,...,0.173332,4,34,22,397.149000,-86.851000,0.026574,0.001068,-0.836604,-0.836604
4,5,259584,0,0,327.0,1,1,5,0.335385,0,...,0.173332,4,19,20,397.149000,70.149000,0.026574,0.000146,-0.332471,-0.332471
5,5,512551,0,0,324.0,0,1,6,0.332308,0,...,-0.320233,3,47,44,346.003970,22.003970,0.028636,0.008852,-1.745887,-1.745887
6,5,3392280,0,0,235.0,1,1,7,0.241026,0,...,-0.320233,3,39,47,346.003970,111.003970,0.028636,0.017186,-0.572766,-0.572766
7,5,242706,0,0,314.0,0,1,8,0.322051,0,...,0.073995,4,12,12,485.716843,171.716843,0.023606,-0.010343,0.865444,NaN
8,5,276195,0,0,377.0,1,1,9,0.386667,0,...,0.673332,4,28,29,397.149000,20.149000,0.026574,0.014270,-0.446672,-0.446672
9,5,3265300,0,0,193.0,0,0,10,0.197949,0,...,-0.884820,3,36,36,400.365371,207.365371,0.033202,-0.029492,1.255329,1.255329


In [35]:
df

,srch_id,prop_key,rank_0,rank_1
0,-1046322713,257690,NaN,NaN
1,-1046322713,3066218,NaN,NaN
2,-1046322713,2271987,NaN,NaN
3,-1046322713,3308025,NaN,NaN
4,-1046322713,3222046,NaN,NaN
...,...,...,...,...
166993,964783409,249748,asb,asb
166994,964783409,255778,asb,asb
166995,964783409,244678,asb,asb
166996,964783409,259221,asb,asb


In [10]:
# XGBRanker # objective='rank:ndcg'
group = X_train.groupby('srch_id').size().values
eval_group = [X_train.groupby('srch_id').size().values]
param = [('max_depth', 7)
         # 'subsample': 0.4,
         ## 'reg_lambda': 1.5,
         # 'reg_alpha': 0,
         # 'n_estimators': 1000,
         # 'min_child_weight': 1,
          
         ## 'learning_rate': 0.01,
          #'gamma': 0.3,
          #'colsample_bytree': 0.6,
          #'eta':0.3,
          #'objective':'rank:pairwise', 
          #'num_round':10,
          #'eval_metric':'ndcg@50',
          #'random_state':random_state
         ]




xgb=XGBRanker(objective='rank:pairwise',eval_metric='ndcg@50',num_boost_round=4, max_depth=10, learning_rate=0.01, n_estimators=1000, n_jobs=-1, colsample_bytree=0.6, subsample=0.4, eta=0.3, random_state=random_state)


xgb_model = train_xgb_model(xgb, X_train, y_train, Dataforevaluating, group=group, eval_group=eval_group)

********************TRAIN********************
train avg precision: 0.27
train avg ndcg@50: 0.73
********************TEST********************
The NDCG for the test Dataset is: 0.5064182025369334


##### Rank & make submission

In [11]:
mytestdata = pd.read_csv(r'Data/mytest.csv')
mytestdata= mytestdata.sort_values(by=['srch_id'], ascending=[False])
mytestdata

,srch_visitor_visit_nbr,prop_travelad_bool,prop_dotd_bool,prop_price_with_discount_usd,prop_imp_drr,prop_brand_bool,search_rank,price_ratio,minornot,market_Rate,...,Diff_PropSTR_SubmarketSTR,starrating_floor,Rank_by_StarRating_&_price,Rank_by_StarRating_&_Bookingrate,Submarket_price_mean,Diff_PropPrice_SubmarketPrice,Submarket_Br_mean,Diff_PropSTR_SubmarketSTR_BR,srch_id,prop_key
73125,9,0,0,232.00,1,1,21,0.464930,0,0.030488,...,0.146650,4,2,29,278.932073,46.932073,0.030782,0.023968,2146902698,854345
11337,9,0,0,175.00,0,0,28,0.350701,0,0.033202,...,-0.884084,3,35,38,388.148623,213.148623,0.033309,-0.007090,2146902698,244678
6987,9,0,0,258.00,1,1,38,0.517034,0,0.033202,...,0.115916,4,6,9,388.148623,130.148623,0.033309,-0.005455,2146902698,255778
172792,9,0,0,278.00,1,0,40,0.557114,0,0.012460,...,-0.161697,4,10,21,467.714299,189.714299,0.012470,-0.001080,2146902698,270744
63573,9,0,0,220.00,1,0,5,0.440882,0,0.028636,...,-0.820274,2,51,50,340.352190,120.352190,0.028591,-0.021932,2146902698,250312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78731,44,0,0,202.05,1,0,29,0.487514,0,0.027506,...,0.036187,3,24,22,217.818105,15.768105,0.027380,0.008461,-2147403968,481812
86251,44,0,0,117.86,1,1,32,0.284377,0,0.033569,...,-0.159808,3,13,26,261.878809,144.018809,0.033301,0.020589,-2147403968,271366
93663,44,0,0,176.14,1,0,23,0.424997,0,0.040299,...,0.034448,3,20,25,242.886250,66.746250,0.040117,0.027097,-2147403968,3076641
81254,44,0,0,323.79,0,1,4,0.781252,0,0.024236,...,-0.493810,3,32,23,365.280515,41.490515,0.024032,0.006302,-2147403968,248136


In [12]:
final_df = mytestdata[['srch_id', 'prop_key']].reset_index(drop=True).copy() 

final_df

,srch_id,prop_key
0,2146902698,854345
1,2146902698,244678
2,2146902698,255778
3,2146902698,270744
4,2146902698,250312
...,...,...
351140,-2147403968,481812
351141,-2147403968,271366
351142,-2147403968,3076641
351143,-2147403968,248136


In [13]:
srch_id = mytestdata['srch_id'].unique()
srch_id

array([ 2146902698,  2146811608,  2146615716, ..., -2145205355,
       -2145422147, -2147403968], dtype=int64)

In [14]:
mytestdata= mytestdata.set_index('srch_id')
mytestdata

,srch_visitor_visit_nbr,prop_travelad_bool,prop_dotd_bool,prop_price_with_discount_usd,prop_imp_drr,prop_brand_bool,search_rank,price_ratio,minornot,market_Rate,...,prop_meanPrice_rank,Diff_PropSTR_SubmarketSTR,starrating_floor,Rank_by_StarRating_&_price,Rank_by_StarRating_&_Bookingrate,Submarket_price_mean,Diff_PropPrice_SubmarketPrice,Submarket_Br_mean,Diff_PropSTR_SubmarketSTR_BR,prop_key
srch_id,,,,,,,,,,,,,,,,,,,,,
2146902698,9,0,0,232.00,1,1,21,0.464930,0,0.030488,...,48,0.146650,4,2,29,278.932073,46.932073,0.030782,0.023968,854345
2146902698,9,0,0,175.00,0,0,28,0.350701,0,0.033202,...,50,-0.884084,3,35,38,388.148623,213.148623,0.033309,-0.007090,244678
2146902698,9,0,0,258.00,1,1,38,0.517034,0,0.033202,...,4,0.115916,4,6,9,388.148623,130.148623,0.033309,-0.005455,255778
2146902698,9,0,0,278.00,1,0,40,0.557114,0,0.012460,...,42,-0.161697,4,10,21,467.714299,189.714299,0.012470,-0.001080,270744
2146902698,9,0,0,220.00,1,0,5,0.440882,0,0.028636,...,38,-0.820274,2,51,50,340.352190,120.352190,0.028591,-0.021932,250312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-2147403968,44,0,0,202.05,1,0,29,0.487514,0,0.027506,...,22,0.036187,3,24,22,217.818105,15.768105,0.027380,0.008461,481812
-2147403968,44,0,0,117.86,1,1,32,0.284377,0,0.033569,...,36,-0.159808,3,13,26,261.878809,144.018809,0.033301,0.020589,271366
-2147403968,44,0,0,176.14,1,0,23,0.424997,0,0.040299,...,23,0.034448,3,20,25,242.886250,66.746250,0.040117,0.027097,3076641


In [15]:
mydata.describe()

,srch_id,srch_visitor_visit_nbr,prop_key,prop_travelad_bool,prop_dotd_bool,prop_price_with_discount_usd,prop_imp_drr,prop_brand_bool,prop_booking_bool,search_rank,...,prop_Price_rank,prop_meanPrice_rank,Diff_PropSTR_SubmarketSTR,starrating_floor,Rank_by_StarRating_&_price,Rank_by_StarRating_&_Bookingrate,Submarket_price_mean,Diff_PropPrice_SubmarketPrice,Submarket_Br_mean,Diff_PropSTR_SubmarketSTR_BR
count,8.410560e+05,841056.000000,8.410560e+05,841056.000000,841056.000000,8.410560e+05,841056.000000,841056.000000,841056.000000,841056.000000,...,841056.000000,841056.000000,8.410560e+05,841056.000000,841056.000000,841056.000000,841056.000000,8.410560e+05,841056.000000,8.410560e+05
mean,1.722515e+07,12.326331,8.185409e+05,0.012222,0.003809,3.044550e+02,0.426028,0.664123,0.028549,23.264252,...,19.754562,23.262585,3.941218e-11,3.476422,23.264252,23.264252,304.455029,2.447903e-09,0.028549,1.751715e-10
std,1.231757e+09,31.789589,1.141239e+06,0.109874,0.061603,4.205210e+03,0.494498,0.472296,0.166534,14.975908,...,12.478882,14.974969,7.424305e-01,0.896356,14.975908,14.975908,164.111349,4.202007e+03,0.008849,2.429999e-02
min,-2.147275e+09,1.000000,2.407460e+05,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1.000000,...,1.000000,1.000000,-4.246078e+00,0.000000,1.000000,1.000000,46.199099,-1.086715e+06,0.000000,-9.868421e-01
25%,-1.034005e+09,1.000000,2.559520e+05,0.000000,0.000000,9.902000e+01,0.000000,0.000000,0.000000,9.000000,...,9.000000,9.000000,-3.565113e-01,3.000000,9.000000,9.000000,177.177948,3.526847e+00,0.023262,-1.034288e-02
50%,3.174438e+07,3.000000,2.777420e+05,0.000000,0.000000,1.890000e+02,0.000000,1.000000,0.000000,22.000000,...,19.000000,22.000000,7.485052e-02,4.000000,22.000000,22.000000,310.476271,7.700397e+01,0.027273,2.699143e-03
75%,1.081646e+09,10.000000,4.781330e+05,0.000000,0.000000,2.990000e+02,1.000000,1.000000,0.000000,36.000000,...,30.000000,36.000000,3.812291e-01,4.000000,36.000000,36.000000,400.365371,1.559126e+02,0.035668,1.470108e-02
max,2.147266e+09,1082.000000,3.949856e+06,1.000000,1.000000,1.087117e+06,1.000000,1.000000,1.000000,54.000000,...,53.000000,54.000000,3.762963e+00,5.000000,54.000000,54.000000,10101.900000,5.840522e+03,1.000000,2.000000e-01


In [16]:
mytestdata.describe()

,srch_visitor_visit_nbr,prop_travelad_bool,prop_dotd_bool,prop_price_with_discount_usd,prop_imp_drr,prop_brand_bool,search_rank,price_ratio,minornot,market_Rate,...,prop_meanPrice_rank,Diff_PropSTR_SubmarketSTR,starrating_floor,Rank_by_StarRating_&_price,Rank_by_StarRating_&_Bookingrate,Submarket_price_mean,Diff_PropPrice_SubmarketPrice,Submarket_Br_mean,Diff_PropSTR_SubmarketSTR_BR,prop_key
count,351145.000000,351145.000000,351145.000000,351145.000000,351145.000000,351145.000000,351145.000000,351145.000000,351145.000000,351145.000000,...,351145.000000,351145.000000,351145.000000,351145.000000,351145.000000,351145.000000,351145.000000,351145.000000,351145.000000,3.511450e+05
mean,12.214655,0.012616,0.004141,284.777886,0.425015,0.662188,23.252335,0.445080,0.030731,0.028523,...,23.236828,-0.003773,3.470096,23.236996,23.236996,298.570270,13.792384,0.028509,0.000042,8.160525e+05
std,31.553132,0.111610,0.064215,2810.383920,0.494346,0.472965,14.987500,0.238775,0.172588,0.008843,...,14.983420,0.744283,0.897943,14.983564,14.983564,160.678641,2803.110281,0.008832,0.024209,1.137536e+06
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,-4.227766,0.000000,1.000000,1.000000,45.391278,-334051.935100,0.000000,-0.980914,2.407460e+05
25%,1.000000,0.000000,0.000000,99.000000,0.000000,0.000000,9.000000,0.254682,0.000000,0.023262,...,9.000000,-0.357308,3.000000,9.000000,9.000000,175.340493,-3.045471,0.023269,-0.010404,2.561330e+05
50%,3.000000,0.000000,0.000000,189.000000,0.000000,1.000000,22.000000,0.417933,0.000000,0.027273,...,22.000000,0.077715,4.000000,22.000000,22.000000,292.304352,65.939601,0.027099,0.002779,2.777420e+05
75%,10.000000,0.000000,0.000000,299.000000,1.000000,1.000000,36.000000,0.603025,0.000000,0.035668,...,36.000000,0.382441,4.000000,36.000000,36.000000,389.952228,140.844738,0.035699,0.014706,4.781330e+05
max,1070.000000,1.000000,1.000000,334780.000000,1.000000,1.000000,54.000000,1.000000,1.000000,0.285714,...,54.000000,3.798883,5.000000,54.000000,54.000000,1449.430945,1449.430945,0.285714,0.222222,3.949856e+06


In [23]:
#mytestdata.loc[mytestdata.index == 2146902698]

In [19]:
X_train.columns

Index(['srch_visitor_visit_nbr', 'prop_key', 'prop_travelad_bool',
       'prop_dotd_bool', 'prop_price_with_discount_usd', 'prop_imp_drr',
       'prop_brand_bool', 'search_rank', 'price_ratio', 'minornot',
       'market_Rate', 'discountrate', 'Booking_rate', 'prop_starrating_rank',
       'prop_room_capacity_rank', 'prop_review_score_rank',
       'prop_review_count_rank', 'Mean_Price_Prop', 'SD_Price_prop',
       'Median_Price_prop', 'Diff_Price_Prop_mean',
       'prop_discount_history_rate', 'prop_appear_history', 'prop_Price_rank',
       'prop_meanPrice_rank', 'Diff_PropSTR_SubmarketSTR', 'starrating_floor',
       'Rank_by_StarRating_&_price', 'Rank_by_StarRating_&_Bookingrate',
       'Submarket_price_mean', 'Diff_PropPrice_SubmarketPrice',
       'Submarket_Br_mean', 'Diff_PropSTR_SubmarketSTR_BR'],
      dtype='object')

In [21]:
mytestdata=mytestdata[X_train.columns]
mytestdata

,srch_visitor_visit_nbr,prop_key,prop_travelad_bool,prop_dotd_bool,prop_price_with_discount_usd,prop_imp_drr,prop_brand_bool,search_rank,price_ratio,minornot,...,prop_Price_rank,prop_meanPrice_rank,Diff_PropSTR_SubmarketSTR,starrating_floor,Rank_by_StarRating_&_price,Rank_by_StarRating_&_Bookingrate,Submarket_price_mean,Diff_PropPrice_SubmarketPrice,Submarket_Br_mean,Diff_PropSTR_SubmarketSTR_BR
srch_id,,,,,,,,,,,,,,,,,,,,,
2146902698,9,854345,0,0,232.00,1,1,21,0.464930,0,...,37,48,0.146650,4,2,29,278.932073,46.932073,0.030782,0.023968
2146902698,9,244678,0,0,175.00,0,0,28,0.350701,0,...,45,50,-0.884084,3,35,38,388.148623,213.148623,0.033309,-0.007090
2146902698,9,255778,0,0,258.00,1,1,38,0.517034,0,...,31,4,0.115916,4,6,9,388.148623,130.148623,0.033309,-0.005455
2146902698,9,270744,0,0,278.00,1,0,40,0.557114,0,...,26,42,-0.161697,4,10,21,467.714299,189.714299,0.012470,-0.001080
2146902698,9,250312,0,0,220.00,1,0,5,0.440882,0,...,40,38,-0.820274,2,51,50,340.352190,120.352190,0.028591,-0.021932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-2147403968,44,481812,0,0,202.05,1,0,29,0.487514,0,...,19,22,0.036187,3,24,22,217.818105,15.768105,0.027380,0.008461
-2147403968,44,271366,0,0,117.86,1,1,32,0.284377,0,...,32,36,-0.159808,3,13,26,261.878809,144.018809,0.033301,0.020589
-2147403968,44,3076641,0,0,176.14,1,0,23,0.424997,0,...,23,23,0.034448,3,20,25,242.886250,66.746250,0.040117,0.027097


In [22]:
#srch_id=[2146811608]

rank=[]
for search_Id in srch_id:
    final_df.loc[final_df['srch_id'] == search_Id, 'rank'] = xgb_model.predict(mytestdata.loc[mytestdata.index == search_Id])
#final_df['rank'] = xgb_model.predict(mytestdata)

In [23]:
submission_df = final_df[['srch_id', 'prop_key', 'rank']].sort_values(by=['srch_id', 'rank'], ascending=[False, False])\
                .drop('rank', axis=1).set_index('srch_id')

In [24]:
submission_df.head()

,prop_key
srch_id,
2146902698,3265300
2146902698,263860
2146902698,259283
2146902698,255778
2146902698,3083617


In [25]:
submission_df.to_csv('submissionmahmood2.csv')